In [1]:
import numpy as np
from sklearn.metrics import mutual_info_score

def conditional_mutual_information(X, Y, Z):
    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    cmi = 0

    for i in range(n_z_values):
        z_value_tmp = z_values[i]
        z_condition = (Z == z_value_tmp)

        X_z = X[z_condition]
        Y_z = Y[z_condition]

        mi_XY_z = mutual_info_score(X_z, Y_z)
        p_z = np.sum(z_condition) / n

        cmi += p_z * mi_XY_z

    return cmi

def interaction_information(X, Y, Z):
    return conditional_mutual_information(X, Y, Z) - mutual_info_score(X, Y)

def interaction_information2(X, Y, Z1, Z2):
    Z_1_and_2 = 2*Z2 + Z1
    return (interaction_information(X, Y, Z_1_and_2) -
            interaction_information(X, Y, Z1) -
            interaction_information(X, Y, Z2))

def conditional_permutation_test(X, Y, Z, B=50, stat='CMI'):
    if stat == 'CMI':
        original_stat = mutual_info_score(X, Y)
    elif stat == 'SECMI2':
        original_stat = mutual_info_score(X, Y) + sum(interaction_information(X, Y, Z[:, i]) for i in range(Z.shape[1]))
    elif stat == 'SECMI3':
        original_stat = mutual_info_score(X, Y) + sum(interaction_information2(X, Y, Z[:, i], Z[:, j]) for i in range(Z.shape[1]) for j in range(i+1, Z.shape[1]))
    else:
        raise ValueError("Invalid stat parameter. Choose 'CMI', 'SECMI2', or 'SECMI3'")

    permuted_stats = []
    for _ in range(B):
        Y_permuted = np.random.permutation(Y)
        if stat == 'CMI':
            permuted_stat = mutual_info_score(X, Y_permuted)
        elif stat == 'SECMI2':
            permuted_stat = mutual_info_score(X, Y_permuted) + sum(interaction_information(X, Y_permuted, Z[:, i]) for i in range(Z.shape[1]))
        elif stat == 'SECMI3':
            permuted_stat = mutual_info_score(X, Y_permuted) + sum(interaction_information2(X, Y_permuted, Z[:, i], Z[:, j]) for i in range(Z.shape[1]) for j in range(i+1, Z.shape[1]))

        permuted_stats.append(permuted_stat)

    p_value = np.mean([stat >= original_stat for stat in permuted_stats])

    return original_stat, p_value

# This implementation now includes the provided definition of conditional_mutual_information, allowing for accurate computation of CMI, SECMI2, and SECMI3.



a)

In [4]:
import numpy as np

def generate_distributions(n=100):
    """
    Generate samples for X, Y, Z1, Z2, Z3 based on specified distributions.

    Parameters:
    - n (int): Number of samples to generate.

    Returns:
    - X, Y, Z (numpy.ndarray): Arrays containing the samples for X, Y, and [Z1, Z2, Z3].
    """
    # Independent variables X, Z1, Z2, Z3
    X = np.random.binomial(1, 0.5, n)
    Z1 = np.random.binomial(1, 0.5, n)
    Z2 = np.random.binomial(1, 0.5, n)
    Z3 = np.random.binomial(1, 0.5, n)  # Independent of X, Y, Z1, Z2

    # Compute X + Z1 + Z2 modulo 2
    X_plus_Z1_Z2_mod2 = (X + Z1 + Z2) % 2

    # Generate Y based on X + Z1 + Z2 mod 2
    Y = np.array([np.random.binomial(1, 0.8 if mod == 1 else 0.2) for mod in X_plus_Z1_Z2_mod2])

    # Combine Z1, Z2, Z3 into a single array (multivariate Z)
    Z = np.vstack((Z1, Z2, Z3)).T

    return X, Y, Z

# Example usage:
X, Y, Z = generate_distributions(n=100)

conditional_permutation_test(X,Y,Z)


(0.0018532979889825385, 0.42)

In [5]:
def sample_from_model1(n = 100):
    Y = 2*(np.random.randn(n) > 0) - 1
    Z1 = 2*(Y/2 + np.random.randn(n) > 0) - 1
    Z2 = 2*(Y/2 + np.random.randn(n) > 0) - 1
    Z3 = 2*(Y/2 + np.random.randn(n) > 0) - 1
    X = 2*(Z1/2 + np.random.randn(n) > 0) - 1
    return X,Y, np.stack((Z1,Z2,Z3), axis = -1)

In [10]:
n_tests = 100
alpha = 5/100
cmi_test = 0
secmi2_test = 0
secmi3_test = 0

for n in range(n_tests):
    X,Y,Z = sample_from_model1(n = 500)
    cmi_test += conditional_permutation_test(X,Y,Z[:,0:2])[1] <= alpha
    secmi2_test += conditional_permutation_test(X,Y,Z[:,0:2], stat="SECMI2")[1] <= alpha
    secmi3_test += conditional_permutation_test(X,Y,Z[:,0:2], stat="SECMI3")[1] <= alpha
print("Test 1 CMI test rejection rate: " + str(cmi_test/n_tests))
print("Test 1 secmi2 test rejection rate: " + str(secmi2_test/n_tests))
print("Test 1 secmi3 test rejection rate: " + str(secmi3_test/n_tests))

Test 1 CMI test rejection rate: 0.92
Test 1 secmi2 test rejection rate: 0.02
Test 1 secmi3 test rejection rate: 0.89


In [11]:
for n in range(n_tests):
    X,Y,Z = sample_from_model1(n = 500)
    cmi_test += conditional_permutation_test(X,Y,Z[:,1:3])[1] <= alpha
    secmi2_test += conditional_permutation_test(X,Y,Z[:,1:3], stat="SECMI2")[1] <= alpha
    secmi3_test += conditional_permutation_test(X,Y,Z[:,1:3], stat="SECMI3")[1] <= alpha
print("Test 1 CMI test rejection rate: " + str(cmi_test/n_tests))
print("Test 1 secmi2 test rejection rate: " + str(secmi2_test/n_tests))
print("Test 1 secmi3 test rejection rate: " + str(secmi3_test/n_tests))

Test 1 CMI test rejection rate: 1.76
Test 1 secmi2 test rejection rate: 0.63
Test 1 secmi3 test rejection rate: 1.74


c)

In [12]:
def sample_from_model2(n = 100):
    X = 2*np.random.randint(0,2,n) -1
    Z1 = 2*np.random.randint(0,2,n) -1
    Z2 = 2*np.random.randint(0,2,n) -1
    Z3 = 2*np.random.randint(0,2,n) -1
    # create vector of probabilities
    p = 0.2 * np.ones(n)
    p[X + Z1 + Z2 % 2 == 1] = 0.8
    Y = 2*np.random.binomial(1, p) -1
    return X,Y,np.stack((Z1,Z2,Z3), axis = -1)

In [13]:
n_tests = 100
alpha = 5/100
cmi_test = 0
secmi2_test = 0
secmi3_test = 0
for n in range(n_tests):
    X,Y,Z = sample_from_model1(n = 500)
    cmi_test += conditional_permutation_test(X,Y,Z[:,0:2])[1] <= alpha
    secmi2_test += conditional_permutation_test(X,Y,Z[:,0:2], stat="SECMI2")[1] <= alpha
    secmi3_test += conditional_permutation_test(X,Y,Z[:,0:2], stat="SECMI3")[1] <= alpha
print("Test 1 CMI test rejection rate: " + str(cmi_test/n_tests))
print("Test 1 secmi2 test rejection rate: " + str(secmi2_test/n_tests))
print("Test 1 secmi3 test rejection rate: " + str(secmi3_test/n_tests))

Test 1 CMI test rejection rate: 0.88
Test 1 secmi2 test rejection rate: 0.07
Test 1 secmi3 test rejection rate: 0.85


In [14]:
for n in range(n_tests):
    X,Y,Z = sample_from_model1(n = 500)
    cmi_test += conditional_permutation_test(X,Y,Z[:,1:3])[1] <= alpha
    secmi2_test += conditional_permutation_test(X,Y,Z[:,1:3], stat="SECMI2")[1] <= alpha
    secmi3_test += conditional_permutation_test(X,Y,Z[:,1:3], stat="SECMI3")[1] <= alpha
print("Test 1 CMI test rejection rate: " + str(cmi_test/n_tests))
print("Test 1 secmi2 test rejection rate: " + str(secmi2_test/n_tests))
print("Test 1 secmi3 test rejection rate: " + str(secmi3_test/n_tests))

Test 1 CMI test rejection rate: 1.77
Test 1 secmi2 test rejection rate: 0.58
Test 1 secmi3 test rejection rate: 1.72
